In [ ]:

!pip install -q torch transformers accelerate peft trl bitsandbytes scipy

In [ ]:
!pip install -q -U transformers peft accelerate bitsandbytes trl

In [ ]:
!pip install bitsandbytes==0.41.0


  Using cached bitsandbytes-0.41.0-py3-none-any.whl.metadata (9.8 kB)
Using cached bitsandbytes-0.41.0-py3-none-any.whl (92.6 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.48.2
    Uninstalling bitsandbytes-0.48.2:
      Successfully uninstalled bitsandbytes-0.48.2


In [ ]:

!pip uninstall -y torch torchvision torchaudio bitsandbytes
!pip install -q -U transformers peft accelerate trl
!pip install -q -U bitsandbytes
!pip install torch --pre --extra-index-url https://download.pytorch.org/whl/nightly/cu121

import os
os.kill(os.getpid(), 9)

Found existing installation: torch 2.9.0
Uninstalling torch-2.9.0:
  Successfully uninstalled torch-2.9.0
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Found existing installation: bitsandbytes 0.41.0
Uninstalling bitsandbytes-0.41.0:
  Successfully uninstalled bitsandbytes-0.41.0
ERROR: Operation cancelled by user
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cu121


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:

!pip install -q -U --no-cache-dir bitsandbytes peft accelerate trl transformers
!pip install -q -U torch --pre --extra-index-url https://download.pytorch.org/whl/nightly/cu121

import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install -U "trl>=0.8.6" "transformers>=4.44" "peft>=0.12" "bitsandbytes>=0.43" "accelerate>=1.0"


Antrenament limba romana

In [ ]:
import os
from dataclasses import dataclass
from typing import Dict, List, Optional

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer
from peft import prepare_model_for_kbit_training
FILE_NAME = "boli_antrenament_final.json"
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

raw_ds = load_dataset("json", data_files=FILE_NAME, split="train")
print(f"Loaded {len(raw_ds)} examples.")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def map_to_text(example):
    msgs = example.get("messages", [])
    norm = []
    for m in msgs:
        role = m.get("role", "user")
        if role not in ("system", "user", "assistant"):
            role = "user"
        content = (m.get("content") or "").strip()
        if not content:
            continue
        norm.append({"role": role, "content": content})


    text = tokenizer.apply_chat_template(
        norm,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

ds = raw_ds.map(map_to_text, remove_columns=raw_ds.column_names)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

model = get_peft_model(model, lora_config)
model.gradient_checkpointing_enable()

print(f"Număr de parametri antrenabili: {model.print_trainable_parameters()}")

assistant_trigger = "<|assistant|>:"

from dataclasses import dataclass
from typing import Dict, List, Optional
import torch

ASSISTANT_TRIGGER = "<|assistant|>:"

@dataclass
class TokenizedCompletionOnlyCollator:
    tokenizer: AutoTokenizer
    response_template: str
    max_length: int = 1024
    pad_to_multiple_of: Optional[int] = 8

    def _pad(self, sequences: List[List[int]], pad_id: int) -> torch.Tensor:
        max_len = max(len(s) for s in sequences)
        if self.pad_to_multiple_of:
            rem = max_len % self.pad_to_multiple_of
            if rem != 0:
                max_len += (self.pad_to_multiple_of - rem)
        out = []
        for s in sequences:
            out.append(s + [pad_id] * (max_len - len(s)))
        return torch.tensor(out, dtype=torch.long)

    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]
        if "input_ids" in features[0]:
            input_ids_list = [f["input_ids"] for f in features]
            attn_list      = [f.get("attention_mask", [1]*len(f["input_ids"])) for f in features]

            input_ids = self._pad(input_ids_list, self.tokenizer.pad_token_id)
            attention_mask = self._pad(attn_list, 0)

        else:
            texts = [f["text"] for f in features]
            enc = self.tokenizer(
                texts,
                max_length=self.max_length,
                truncation=True,
                padding=True,
                return_tensors="pt"
            )
            input_ids = enc["input_ids"]
            attention_mask = enc["attention_mask"]

        labels = input_ids.clone()

        templ_ids = self.tokenizer(self.response_template, add_special_tokens=False).input_ids
        tlen = len(templ_ids)

        for i in range(input_ids.size(0)):
            row = input_ids[i].tolist()
            start_idx = -1
            for j in range(0, len(row) - tlen + 1):
                if row[j:j+tlen] == templ_ids:
                    start_idx = j
            if start_idx > 0:
                labels[i, :start_idx] = -100

        labels[attention_mask == 0] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

collator = TokenizedCompletionOnlyCollator(
    tokenizer=tokenizer,
    response_template=ASSISTANT_TRIGGER,
    max_length=1024
)


training_args = TrainingArguments(
    output_dir="./tinyllama_stoma_results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    logging_steps=5,
    save_steps=100,
    save_total_limit=2,
    optim="paged_adamw_32bit",
    fp16=True,
    bf16=False,
    report_to="none",
    max_steps=300,
)
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    peft_config=lora_config,
    data_collator=collator,
)

try:
    trainer.train()
finally:
    trainer.model.save_pretrained("./tinyllama_stoma_final")
    tokenizer.save_pretrained("./tinyllama_stoma_final")
    print("Model salvat în ./tinyllama_stoma_final")


Loaded 30 examples.


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044
Număr de parametri antrenabili: None


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss
5,2.449200
10,2.282200
15,2.104600
20,1.952900
25,1.809500
30,1.654600
35,1.524500
40,1.376300
45,1.262900
50,1.165300


Model salvat în ./tinyllama_stoma_final


In [ ]:
#salvare si descarcare folder
!zip -r /content/tinyllama_stoma_final.zip /content/tinyllama_stoma_final

  adding: content/tinyllama_stoma_final/ (stored 0%)
  adding: content/tinyllama_stoma_final/tokenizer.model (deflated 55%)
  adding: content/tinyllama_stoma_final/special_tokens_map.json (deflated 79%)
  adding: content/tinyllama_stoma_final/README.md (deflated 65%)
  adding: content/tinyllama_stoma_final/adapter_model.safetensors (deflated 8%)
  adding: content/tinyllama_stoma_final/tokenizer.json (deflated 85%)
  adding: content/tinyllama_stoma_final/chat_template.jinja (deflated 60%)
  adding: content/tinyllama_stoma_final/adapter_config.json (deflated 56%)
  adding: content/tinyllama_stoma_final/tokenizer_config.json (deflated 69%)


Testare limba romana

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

BASE_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
ADAPTER_DIR = "/content/tinyllama_stoma_final"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("[INFO] Loading base model...")
base = AutoModelForCausalLM.from_pretrained(BASE_ID, quantization_config=bnb, device_map="auto")

print("[INFO] Loading LoRA adapters...")
model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

#PROMPT
messages = [
    {"role": "system", "content": "Tu ești un pacient care are o problemă dentară. Nu spui diagnosticul."},
    {"role": "user", "content": "Ce vă supără cel mai mult?"}
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

print("\n[INFO] Generating response...\n")
with torch.inference_mode():
    out = model.generate(**inputs, max_new_tokens=150, do_sample=True, temperature=0.7, top_p=0.9)

decoded = tokenizer.decode(out[0], skip_special_tokens=True)
print("OUTPUT:\n", decoded)


[INFO] Loading base model...
[INFO] Loading LoRA adapters...

[INFO] Generating response...

🔹 OUTPUT:
 <|system|>
Tu ești un pacient care are o problemă dentară. Nu spui diagnosticul. 
<|user|>
Ce vă supără cel mai mult? 
<|assistant|>
Am băut un foarte bun de ceptăti, am încercat să ne supără, dar am făcut un test. E către ulcere.


Antrenament limba engleza

In [ ]:
import os
from dataclasses import dataclass
from typing import Dict, List, Optional

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer
from peft import prepare_model_for_kbit_training
FILE_NAME = "boli_antrenament_engleza.json"
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

raw_ds = load_dataset("json", data_files=FILE_NAME, split="train")
print(f"Loaded {len(raw_ds)} examples.")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def map_to_text(example):
    msgs = example.get("messages", [])
    norm = []
    for m in msgs:
        role = m.get("role", "user")
        if role not in ("system", "user", "assistant"):
            role = "user"
        content = (m.get("content") or "").strip()
        if not content:
            continue
        norm.append({"role": role, "content": content})

    text = tokenizer.apply_chat_template(
        norm,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

ds = raw_ds.map(map_to_text, remove_columns=raw_ds.column_names)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

model = get_peft_model(model, lora_config)

model.gradient_checkpointing_enable()

print(f"Număr de parametri antrenabili: {model.print_trainable_parameters()}")

assistant_trigger = "<|assistant|>:"

from dataclasses import dataclass
from typing import Dict, List, Optional
import torch

ASSISTANT_TRIGGER = "<|assistant|>:"

@dataclass
class TokenizedCompletionOnlyCollator:
    tokenizer: AutoTokenizer
    response_template: str
    max_length: int = 1024
    pad_to_multiple_of: Optional[int] = 8

    def _pad(self, sequences: List[List[int]], pad_id: int) -> torch.Tensor:
        max_len = max(len(s) for s in sequences)
        if self.pad_to_multiple_of:
            rem = max_len % self.pad_to_multiple_of
            if rem != 0:
                max_len += (self.pad_to_multiple_of - rem)
        out = []
        for s in sequences:
            out.append(s + [pad_id] * (max_len - len(s)))
        return torch.tensor(out, dtype=torch.long)

    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]:

        if "input_ids" in features[0]:
            input_ids_list = [f["input_ids"] for f in features]
            attn_list      = [f.get("attention_mask", [1]*len(f["input_ids"])) for f in features]

            input_ids = self._pad(input_ids_list, self.tokenizer.pad_token_id)
            attention_mask = self._pad(attn_list, 0)


        else:
            texts = [f["text"] for f in features]
            enc = self.tokenizer(
                texts,
                max_length=self.max_length,
                truncation=True,
                padding=True,
                return_tensors="pt"
            )
            input_ids = enc["input_ids"]
            attention_mask = enc["attention_mask"]

        labels = input_ids.clone()

        templ_ids = self.tokenizer(self.response_template, add_special_tokens=False).input_ids
        tlen = len(templ_ids)


        for i in range(input_ids.size(0)):
            row = input_ids[i].tolist()
            start_idx = -1
            for j in range(0, len(row) - tlen + 1):
                if row[j:j+tlen] == templ_ids:
                    start_idx = j
            if start_idx > 0:
                labels[i, :start_idx] = -100


        labels[attention_mask == 0] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

collator = TokenizedCompletionOnlyCollator(
    tokenizer=tokenizer,
    response_template=ASSISTANT_TRIGGER,
    max_length=1024
)


training_args = TrainingArguments(
    output_dir="./tinyllama_stoma_results_engleza",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    logging_steps=5,
    save_steps=100,
    save_total_limit=2,
    optim="paged_adamw_32bit",
    fp16=True,
    bf16=False,
    report_to="none",
    max_steps=300,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    peft_config=lora_config,
    data_collator=collator,
)

try:
    trainer.train()
finally:
    trainer.model.save_pretrained("./tinyllama_stoma_engleza")
    tokenizer.save_pretrained("./tinyllama_stoma_engleza")
    print("Model salvat în ./tinyllama_stoma_engleza")


Generating train split: 0 examples [00:00, ? examples/s]

Loaded 29 examples.


Map:   0%|          | 0/29 [00:00<?, ? examples/s]

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044
Număr de parametri antrenabili: None


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/29 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/29 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/29 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss
5,1.966900
10,1.770900
15,1.672400
20,1.548900
25,1.424400
30,1.319000


In [ ]:
import torch, os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

BASE_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
ADAPTER_DIR = "/content/tinyllama_stoma_engleza\"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base = AutoModelForCausalLM.from_pretrained(BASE_ID, quantization_config=bnb, device_map="auto")
model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()

def build_patient_prompt(pathology, mf):
    return f"""You are the PATIENT (the assistant). DO NOT reveal your diagnosis.
Pathology: {pathology}

INSTRUCTIONS:
1) Answer naturally, like a real human patient — short, realistic sentences.
2) Stay consistent with the symptoms below.
3) If the doctor tries to guess or ask for the diagnosis, don't confirm it directly. Describe only your symptoms.
4) If asked about something not listed, answer plausibly without contradicting the rest.

SYMPTOMS / CONTEXT:
- Main complaint: {mf.get('chief_complaint','')}
- Pain: {mf.get('pain','')}
- Location: {mf.get('location','')}
- Onset/duration: {mf.get('duration','')}
- Aggravating factors: {mf.get('aggravating','')}
- Relieving factors: {mf.get('relieving','')}
- Appearance: {mf.get('appearance','')}
- History: {mf.get('history','')}
- Other info: {mf.get('extra','')}
"""

manifest = {
    "chief_complaint": "My tooth has turned gray, but it doesn't hurt anymore.",
    "pain": "No pain to cold, heat, or spontaneously. Only slight tenderness when biting or tapping.",
    "location": "Lower right molar.",
    "duration": "Severe pain 2–4 weeks ago, which has completely stopped since then.",
    "appearance": "Tooth looks grayish.",
    "history": "Past episodes of strong spontaneous or night pain.",
}
SYSTEM_PROMPT = build_patient_prompt("Pulpal Necrosis", manifest)

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Good afternoon. What seems to be the problem today?"}
]

def generate_reply(messages, max_new_tokens=150, temperature=0.3, top_p=0.9):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.inference_mode():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=1.1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )


    gen_tokens = output[0, inputs["input_ids"].shape[-1]:]
    reply = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
    return reply

print("🩺 Doctor–Patient Chat started (English). Type /reset or /exit to quit.\n")

assistant_reply = generate_reply(messages)
print(f"Patient: {assistant_reply}\n")
messages.append({"role": "assistant", "content": assistant_reply})

while True:
    user = input("Doctor: ").strip()
    if user.lower() == "/exit":
        break
    if user.lower() == "/reset":
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]
        print("Context reset.\n")
        continue

    messages.append({"role": "user", "content": user})
    assistant_reply = generate_reply(messages)
    print(f"\nPatient: {assistant_reply}\n")
    messages.append({"role": "assistant", "content": assistant_reply})


🩺 Doctor–Patient Chat started (English). Type /reset or /exit to quit.

Patient: My daughter noticed that her tooth on the left side is turning gray. It hasn't been hurting before and it came out yesterday. We didn't have any pain, but it's there.

Doctor: does the pain appear at night?

Patient: No, the pain was only in the daytime. It felt stronger when I tried to sleep last night.

Doctor: It is sensible at cold

Patient: Yes, absolutely. When I put my hand on it, it felt colder than usual.

Doctor: how is the mobility of the tooth is it shaking?

Patient: No, it isn't moving.

Doctor: does she have any cavity or previous oral tratement?

Patient: No, this is her first time visiting.

Doctor: Ok i think she might have Pulpal Necrosis

Patient: Absolutely. The pulp appears to be dying off.



KeyboardInterrupt: Interrupted by user